In [2]:
from pyproj import Transformer
import pandas as pd

In [10]:
# 讀取含有EPSG 3826坐標的檔案，假設檔案名稱為 data.csv
df = pd.read_csv("points_taipei.csv")

In [16]:
# Taipei經緯度
import pandas as pd
from pyproj import Transformer

# 讀取CSV檔案
df = pd.read_csv("points_taipei.csv")

# 定義轉換器，從EPSG 3826 (TWD97) 轉換為 EPSG 4326 (WGS84，即經緯度坐標系統)
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")

# 將每一列的坐標進行轉換
def convert_coordinates(row):
    x1 = row['x1']  # x1 欄位存放了第一組 EPSG 3826 的 X 坐標
    y1 = row['y1']  # y1 欄位存放了第一組 EPSG 3826 的 Y 坐標
    lat1, lon1 = transformer.transform(x1, y1)  # 將第一組坐標轉換為緯度和經度

    x2 = row['x2']  # x2 欄位存放了第二組 EPSG 3826 的 X 坐標
    y2 = row['y2']  # y2 欄位存放了第二組 EPSG 3826 的 Y 坐標
    lat2, lon2 = transformer.transform(x2, y2)  # 將第二組坐標轉換為緯度和經度

    return pd.Series({'Longitude1': lon1, 'Latitude1': lat1, 'Longitude2': lon2, 'Latitude2': lat2})

# 將轉換後的經緯度資料合併到原始資料框中
df[['Longitude1', 'Latitude1', 'Longitude2', 'Latitude2']] = df.apply(convert_coordinates, axis=1)

# 將結果寫入新的 CSV 檔案
df.to_csv("converted_data_taipei.csv", index=False)

print("轉換完成，結果已寫入 converted_data_taipei.csv 檔案中")


轉換完成，結果已寫入 converted_data_taipei.csv 檔案中


In [14]:
# New Taipei City經緯度

# 讀取CSV檔案
new_taipei_city = pd.read_csv("points_new_taipei.csv")

# 定義轉換器，從EPSG 3826 (TWD97) 轉換為 EPSG 4326 (WGS84，即經緯度坐標系統)
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")

# 將每一列的坐標進行轉換
def convert_coordinates(row):
    x1 = row['x1']  # x1 欄位存放了第一組 EPSG 3826 的 X 坐標
    y1 = row['y1']  # y1 欄位存放了第一組 EPSG 3826 的 Y 坐標
    lat1, lon1 = transformer.transform(x1, y1)  # 將第一組坐標轉換為緯度和經度

    x2 = row['x2']  # x2 欄位存放了第二組 EPSG 3826 的 X 坐標
    y2 = row['y2']  # y2 欄位存放了第二組 EPSG 3826 的 Y 坐標
    lat2, lon2 = transformer.transform(x2, y2)  # 將第二組坐標轉換為緯度和經度

    return pd.Series({'Longitude1': lon1, 'Latitude1': lat1, 'Longitude2': lon2, 'Latitude2': lat2})

# 將轉換後的經緯度資料合併到原始資料框中
new_taipei_city[['Longitude1', 'Latitude1', 'Longitude2', 'Latitude2']] = new_taipei_city.apply(convert_coordinates, axis=1)

# 將結果寫入新的 CSV 檔案
new_taipei_city.to_csv("converted_data_new_taipei_city.csv", index=False)

print("轉換完成，結果已寫入 converted_data_new_taipei_city.csv 檔案中")

轉換完成，結果已寫入 converted_data_new_taipei_city.csv 檔案中


In [48]:
# 房屋統計
import pandas as pd
# 讀取 converted_data_taipei.csv 檔案並指定 'Store_Name' 欄位的資料類型為字串
converted_data_df = pd.read_csv("converted_data_new_taipei_city.csv", dtype={"B": str})

# 讀取 geocode_results_台北台灣房屋.xlsx 檔案
geocode_df = pd.read_csv("geocode_results_新北信義房屋.csv", encoding='utf-8-sig')

# 定義尋找對應方格索引的函數
def find_point(row):
    x = row['X']
    y = row['Y']
    for index, point_row in converted_data_df.iterrows():
        if (point_row['Lon1'] <= x <= point_row['Lon2'] or
            point_row['Lon2'] <= x <= point_row['Lon1']) and \
           (point_row['Lat1'] <= y <= point_row['Lat2'] or
            point_row['Lat2'] <= y <= point_row['Lat1']):
            return index
    return None

# 將 geocode_df 中的 X、Y 新增到 converted_data_df 對應方格的 X、Y 欄位中
for index, row in geocode_df.iterrows():
    point_index = find_point(row)
    if point_index is not None:
        converted_data_df.at[point_index, 'X'] = row['X']
        converted_data_df.at[point_index, 'Y'] = row['Y']
        
        # 獲取已存在的 門市名稱
        existing_store = converted_data_df.at[point_index, 'B']
        # 如果已存在的 門市名稱 不為空，則將新的 門市名稱 附加到已存在的 門市名稱 後面，並使用分號分隔
        if pd.notnull(existing_store):
            new_store = f"{existing_store};{row['門市名稱']}"
        else:
            new_store = row['門市名稱']
        converted_data_df.at[point_index, 'B'] = new_store

# 將結果寫入 converted_data_taipei.csv 檔案中
converted_data_df.to_csv("converted_data_new_taipei_city.csv", index=False,encoding='utf-8-sig')

print("已寫入 converted_data_new_taipei_city.csv 信義房屋 檔案中")


C:\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


已寫入 converted_data_new_taipei_city.csv 信義房屋 檔案中
